In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import numpy as np
import warnings



warnings.simplefilter(action="ignore", category='all')

In [2]:
list_symbols = pd.read_excel('C:/projeto_python/ibrx.xls')['Código'].values
list_symbols

array(['RRRP3', 'ALPA4', 'ABEV3', 'AMER3', 'ASAI3', 'AZUL4', 'B3SA3',
       'BIDI11', 'BPAN4', 'BBSE3', 'BRML3', 'BBDC3', 'BBDC4', 'BRAP4',
       'BBAS3', 'BRKM5', 'BRFS3', 'BPAC11', 'CRFB3', 'CCRO3', 'CMIG4',
       'CESP6', 'CIEL3', 'COGN3', 'CPLE6', 'CSAN3', 'CPFE3', 'CMIN3',
       'CVCB3', 'CYRE3', 'DXCO3', 'ECOR3', 'ELET3', 'ELET6', 'EMBR3',
       'ENBR3', 'ENGI11', 'ENEV3', 'EGIE3', 'EQTL3', 'EZTC3', 'FLRY3',
       'GGBR4', 'GOAU4', 'GOLL4', 'NTCO3', 'SOMA3', 'HAPV3', 'HYPE3',
       'IGTI11', 'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LIGT3',
       'RENT3', 'LCAM3', 'LWSA3', 'AMAR3', 'LREN3', 'MGLU3', 'MRFG3',
       'CASH3', 'BEEF3', 'MOVI3', 'MRVE3', 'MULT3', 'PCAR3', 'PETR3',
       'PETR4', 'PRIO3', 'PETZ3', 'PSSA3', 'POSI3', 'QUAL3', 'RADL3',
       'RDOR3', 'RAIL3', 'SBSP3', 'SANB11', 'STBP3', 'CSNA3', 'SLCE3',
       'SULA11', 'SUZB3', 'TAEE11', 'VIVT3', 'TIMS3', 'TOTS3', 'UGPA3',
       'USIM5', 'VALE3', 'VIIA3', 'VBBR3', 'WEGE3', 'YDUQ3'], dtype=object)

In [13]:
list_melhores = []
list_relacao_mm = []
list_precos = []
for ativo in list_symbols:
    soma_fundamentos = 0
    dividend_yield = 0
    try:
        chamada_api = yf.Ticker(ativo+'.SA')
        hist = chamada_api.history(period='30mo',interval = '1wk')
        hist_precos = hist['Close']
        hist_dividendos = hist['Dividends'].tz_convert(None)
        hist_dividendos_ultimo_ano = hist_dividendos.loc[datetime(2022,1,1):datetime(2023,12,31)]
    except:
        continue
    

    dividend_yield = (hist_dividendos_ultimo_ano.sum()/2)/hist_precos[-1] if hist_precos[-1] > 0 else 0
    
    if dividend_yield > 0.03:
        balanco_anual = chamada_api.balance_sheet
        balanco_trimestral = chamada_api.quarterly_balance_sheet
        dre_anual = chamada_api.financials
        dfc_anual = chamada_api.cashflow
        dre_trimestral = chamada_api.quarterly_financials
        dfc_trimestral = chamada_api.quarterly_cashflow
        
        try:
            if dre_anual.keys()[0].year >= 0:
                balanco = balanco_anual
                dre = dre_anual
                dfc = dfc_anual
                variacao_receita = int(dre.loc['Total Revenue'][0] > dre.loc['Total Revenue'][1])
                variacao_ebit = int(dre.loc['EBIT'][0] > dre.loc['EBIT'][1])
                variacao_caixa = int(dfc.loc['Changes In Cash'][0] > dfc.loc['Changes In Cash'][1])
                variacao_divida = int(balanco.loc['Current Liabilities'][0] < balanco.loc['Current Liabilities'][1])
                
                soma_fundamentos = variacao_caixa + variacao_ebit + variacao_receita + variacao_divida

                if soma_fundamentos >= 2:
                    print(ativo,variacao_divida,variacao_caixa,variacao_ebit,variacao_receita,dre_anual.keys()[0])
                    list_melhores.append(ativo)
                    list_relacao_mm.append(abs(hist_precos[-1] / np.mean(hist_precos[-40:])-1))
                    list_precos.append(round(hist_precos[-1],2))
            else:
                continue
        except:
            print(ativo)

ABEV3 0 1 1 1 2022-12-31 00:00:00
AMER3 0 1 0 1 2022-12-31 00:00:00
B3SA3 1 1 1 1 2022-12-31 00:00:00


BIDI11.SA: No data found, symbol may be delisted


BBSE3


BRML3.SA: No data found, symbol may be delisted


BBDC3
BBDC4
BRAP4
BBAS3
BRKM5 1 1 0 0 2022-12-31 00:00:00
CMIG4 0 1 1 1 2022-12-31 00:00:00


...

LREN3 1 0 1 1 2022-12-31 00:00:00
MRFG3 0 1 0 1 2022-12-31 00:00:00
BEEF3 0 0 1 1 2022-12-31 00:00:00
MOVI3 0 1 1 1 2022-12-31 00:00:00
MULT3 0 1 1 1 2022-12-31 00:00:00
PETR3 0 0 1 1 2022-12-31 00:00:00
PETR4 0 0 1 1 2022-12-31 00:00:00
PSSA3
POSI3 1 1 1 1 2022-12-31 00:00:00
SANB11
STBP3 0 0 1 1 2022-12-31 00:00:00
SLCE3 0 1 1 1 2022-12-31 00:00:00


SULA11.SA: No data found, symbol may be delisted


SUZB3 0 0 1 1 2022-12-31 00:00:00
TAEE11 1 1 0 0 2022-12-31 00:00:00
VIVT3 1 0 1 1 2022-12-31 00:00:00


VIIA3.SA: No data found, symbol may be delisted


VBBR3 0 1 0 1 2022-12-31 00:00:00


In [14]:
carteira_df = pd.DataFrame({'MM':list_relacao_mm,'Preços':list_precos}, index=list_melhores).sort_values(by = 'MM',ascending=True)

carteira_df

,MM,Preços
ABEV3,0.012873,13.62
GGBR4,0.024896,22.91
KLBN11,0.028741,22.07
LREN3,0.033716,16.99
CMIG4,0.041009,11.87
SLCE3,0.046525,18.13
B3SA3,0.062155,13.98
HYPE3,0.070907,35.27
GOAU4,0.072643,10.54
MOVI3,0.075867,11.81


In [16]:
lista_drop = ['PETR3']

carteira_limpa = carteira_df.drop(lista_drop)


carteira_limpa

,MM,Preços
ABEV3,0.012873,13.62
GGBR4,0.024896,22.91
KLBN11,0.028741,22.07
LREN3,0.033716,16.99
CMIG4,0.041009,11.87
SLCE3,0.046525,18.13
B3SA3,0.062155,13.98
HYPE3,0.070907,35.27
GOAU4,0.072643,10.54
MOVI3,0.075867,11.81


In [17]:
list_pesos = [(1+x)/(carteira_limpa['MM'].sum()+len(carteira_limpa['MM'].values)) for x in carteira_limpa['MM'].values]
list_pesos.reverse()

list_pesos

[0.04087630301794781,
 0.0403419301003681,
 0.040276803847062524,
 0.03985098519422992,
 0.03793973843238144,
 0.03734660443229509,
 0.036483624100744926,
 0.03635116366638803,
 0.03632419834708356,
 0.036143566512776344,
 0.03608871993643839,
 0.03587022105933694,
 0.03579187835313767,
 0.035080901086797234,
 0.03495493942105636,
 0.03493429444504728,
 0.03474899653427741,
 0.0345760788228282,
 0.034530546828336596,
 0.03442708958558617,
 0.034371348496550054,
 0.03409044000569953,
 0.033588800401762015,
 0.03341176405058153,
 0.033177699296850126,
 0.03301801258252453,
 0.03289461914047909,
 0.03250873230143315]

In [18]:
carteira_limpa['Pesos'] = list_pesos
carteira_limpa

,MM,Preços,Pesos
ABEV3,0.012873,13.62,0.040876
GGBR4,0.024896,22.91,0.040342
KLBN11,0.028741,22.07,0.040277
LREN3,0.033716,16.99,0.039851
CMIG4,0.041009,11.87,0.037940
SLCE3,0.046525,18.13,0.037347
B3SA3,0.062155,13.98,0.036484
HYPE3,0.070907,35.27,0.036351
GOAU4,0.072643,10.54,0.036324
MOVI3,0.075867,11.81,0.036144


In [19]:
patrimonio_inicial = 10000

carteira_limpa['Lote'] = round((carteira_limpa['Pesos'] * patrimonio_inicial/carteira_limpa['Preços'])/100,2)*100
carteira_limpa

,MM,Preços,Pesos,Lote
ABEV3,0.012873,13.62,0.040876,30.0
GGBR4,0.024896,22.91,0.040342,18.0
KLBN11,0.028741,22.07,0.040277,18.0
LREN3,0.033716,16.99,0.039851,23.0
CMIG4,0.041009,11.87,0.037940,32.0
SLCE3,0.046525,18.13,0.037347,21.0
B3SA3,0.062155,13.98,0.036484,26.0
HYPE3,0.070907,35.27,0.036351,10.0
GOAU4,0.072643,10.54,0.036324,34.0
MOVI3,0.075867,11.81,0.036144,31.0


In [20]:

carteira_final = carteira_limpa[['Lote','Preços']]
carteira_final

,Lote,Preços
ABEV3,30.0,13.62
GGBR4,18.0,22.91
KLBN11,18.0,22.07
LREN3,23.0,16.99
CMIG4,32.0,11.87
SLCE3,21.0,18.13
B3SA3,26.0,13.98
HYPE3,10.0,35.27
GOAU4,34.0,10.54
MOVI3,31.0,11.81
